# Classification of Email Types

This script was used to label each email as being either a conference posting, job ad, or a regular communication. It uses a basic linear classifier with SGCD learning. I took it almost verbatim from examples :)

In [1]:
import os
import codecs
import csv
import random
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn import metrics

In [ ]:
attachments = {}
## Use these to get initial email_classification file
#with open('results/email_classification.csv', 'w', newline='') as out:
    #wr = csv.writer(out, quoting=csv.QUOTE_ALL)

files = os.listdir('data/cleaned_attachments/')
data_names = []
data = []
for attachment in files:
    if attachment.endswith('.txt'):
        with codecs.open('data/cleaned_attachments/'+attachment, encoding='utf-8') as attachment_data:
            data_names.append(attachment)
            data.append(attachment_data.read())
            ## Also uncomment these to generate the file
            #short = data[:100].replace('\n', ' ').replace('\r', ' ')
            #wr.writerow(["", attachment, short])
data_names = pd.Series(data_names)
data = pd.Series(data)
paired = pd.Series(dict(zip(data_names, data)))

In [ ]:
training = pd.read_csv('results/email_classification_post.csv',
                       names=['Label', 'ID', 'Text'],
                       index_col='ID',
                       header=None)
del training['Text']
training = training.dropna(subset=['Label'])
training['text'] = paired

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5, random_state=42)),
])
text_clf = text_clf.fit(training.text, training.Label)

In [ ]:
docs_new = paired.drop(training.index)

predicted = text_clf.predict(docs_new)
predicted_paired = pd.Series(predicted, index=docs_new.index)
for doc, category in list(zip(docs_new.index, predicted))[:50]:
    print('%r => %s' % (doc, category))
predicted_paired.to_csv('data/labeled_emails.csv')

In [ ]:
# Test
tested = text_clf.predict(training.text)
print(np.mean(tested == training.Label))

In [ ]:
print(metrics.classification_report(training.Label, tested))
print(metrics.confusion_matrix(training.Label, tested))

I was stunned at how well this ended up working.